In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall, AUC, Accuracy
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
os.environ['CUDA_VISIBLE_DEVICES'] = '1' 

2024-06-07 14:02:10.413357: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 14:02:10.452212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 14:02:10.452239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 14:02:10.453190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-07 14:02:10.459919: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dataset_dir = "101_ObjectCategories"
test_img_path = "/data/selvaraju/Robotic_project/Lab_pictures"

images = []
labels = []
test_images = []

In [3]:
for subclass in os.listdir(dataset_dir):
    subclass_path = os.path.join(dataset_dir, subclass)
    if os.path.isdir(subclass_path):
        for filename in os.listdir(subclass_path):
            image_path = os.path.join(subclass_path, filename)
            image = imread(image_path)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            image = image / 255.0
            images.append(image)
            labels.append(subclass)

In [4]:
for filename in os.listdir(test_img_path):
    img_path = os.path.join(test_img_path, filename)
    if img_path.endswith(('png', 'jpg', 'jpeg')):  
        test_image = load_img(img_path, target_size=(224, 224))  
        test_image = img_to_array(test_image)
        test_image = test_image.astype("float32") / 255.0 
        test_images.append(test_image)

test_images = np.array(test_images)


In [5]:
print("Number of images: ", len(test_images))

Number of images:  11


In [6]:
images = np.array([image for image in images if image.shape == images[0].shape])
labels = np.array(labels)

In [7]:
print("Number of images:", len(images))
print("Number of labels:", len(labels))
if len(images) > len(labels):
    images = images[:len(labels)]
elif len(labels) > len(images):
    labels = labels[:len(images)]
images, labels = shuffle(images, labels, random_state=42)


Number of images: 9145
Number of labels: 9145


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2024-06-07 14:03:05.359680: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 14:03:05.399838: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 14:03:05.400036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = tf.keras.utils.to_categorical(labels_encoded)

In [10]:
datagen = ImageDataGenerator(validation_split=0.2)

train_gen = datagen.flow(images, labels_categorical, subset='training', batch_size=32)
val_gen = datagen.flow(images, labels_categorical, subset='validation', batch_size=32)

In [65]:
input_shape = (224, 224, 3)
reg = l2(0.0001)
num_classes = len(label_encoder.classes_)

In [66]:
def create_model(input_shape, reg, num_classes):
    base_model = MobileNetV2(input_shape= input_shape, include_top=False, weights='imagenet')
    for layers in base_model.layers:
        layers.trainable = False

    x = base_model.output

    x = GlobalAveragePooling2D()(x)
 
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    
    model.summary()
    
    return model

In [67]:
model = create_model(input_shape, reg, num_classes)


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_12[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']     

In [70]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['Accuracy'])

In [71]:
history = model.fit(train_gen, validation_data=val_gen, epochs=20, callbacks=[ModelCheckpoint('model_trans.h5', save_best_only=True), EarlyStopping(patience=3)])

Epoch 1/20
229/229 [==============================] - 9s 30ms/step - loss: 0.0109 - Accuracy: 0.9999 - val_loss: 0.2672 - val_Accuracy: 0.9202
Epoch 2/20
229/229 [==============================] - 6s 24ms/step - loss: 0.0094 - Accuracy: 0.9999 - val_loss: 0.2712 - val_Accuracy: 0.9185
Epoch 3/20
229/229 [==============================] - 6s 25ms/step - loss: 0.0084 - Accuracy: 0.9999 - val_loss: 0.2687 - val_Accuracy: 0.9191
Epoch 4/20
229/229 [==============================] - 6s 27ms/step - loss: 0.0076 - Accuracy: 0.9999 - val_loss: 0.2652 - val_Accuracy: 0.9180
Epoch 5/20
229/229 [==============================] - 6s 26ms/step - loss: 0.0069 - Accuracy: 0.9999 - val_loss: 0.2680 - val_Accuracy: 0.9180
Epoch 6/20
229/229 [==============================] - 7s 29ms/step - loss: 0.0063 - Accuracy: 0.9999 - val_loss: 0.2675 - val_Accuracy: 0.9207
Epoch 7/20
229/229 [==============================] - 6s 26ms/step - loss: 0.0058 - Accuracy: 0.9999 - val_loss: 0.2688 - val_Accuracy: 0.9202

In [72]:
prediction = model.predict(test_images)
predicted_classes = np.argmax(prediction, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)   
print(predicted_labels)

1/1 [==============================] - 1s 730ms/step
['wheelchair' 'BACKGROUND_Google' 'BACKGROUND_Google' 'anchor'
 'BACKGROUND_Google' 'BACKGROUND_Google' 'BACKGROUND_Google' 'Faces'
 'BACKGROUND_Google' 'BACKGROUND_Google' 'BACKGROUND_Google']
